In [1]:
import boto3
from epochai.airtable.models import BenchmarkRun
from inspect_ai.log import read_eval_log
import pandas as pd

In [2]:
# Verify AWS credentials are working
session = boto3.Session()
credentials = session.get_credentials()
if credentials is None:
    print("No AWS credentials found!")
else:
    print("AWS credentials found!")

AWS credentials found!


In [3]:
# Get the runs and read the log header
runs = BenchmarkRun.all()
runs[0].logs

'https://epoch-benchmarks-production-private.s3.us-east-2.amazonaws.com/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval'

In [4]:
len(runs)

135

In [5]:
# Convert the HTTPS URL to an S3 path
def convert_to_s3_path(url):
    # Example input: https://epoch-benchmarks-production-private.s3.us-east-2.amazonaws.com/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval
    # Example output: s3://epoch-benchmarks-production-private/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval
    from urllib.parse import urlparse
    parsed = urlparse(url)
    bucket = parsed.netloc.split('.')[0]
    path = parsed.path.lstrip('/')
    return f"s3://{bucket}/{path}"

# Try reading with the S3 path
s3_path = convert_to_s3_path(runs[0].logs)
print(f"Converting {runs[0].logs} to {s3_path}")

Converting https://epoch-benchmarks-production-private.s3.us-east-2.amazonaws.com/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval to s3://epoch-benchmarks-production-private/inspect_ai_logs/nzy4SGSeKGzQSKJLWExKpV.eval


In [17]:
run_idx = 3
log_header = read_eval_log(convert_to_s3_path(runs[run_idx].logs), header_only=True)
log_header

EvalLog(version=2, status='success', eval=EvalSpec(run_id='4DWnpJowAfeQDHc2xDNy2C', created='2025-02-13T17:08:02+00:00', task='MATH level 5', task_id='ZAhShmdDBAB9WnsZdx5qHf', task_version=0, task_file='bench/task/hendrycks_math/__init__.py', task_attribs={}, task_args={}, solver=None, solver_args=None, tags=None, dataset=EvalDataset(name='tadamcz/hendrycks___competition_math', location='tadamcz/hendrycks___competition_math', samples=1324, sample_ids=[1, 2, 12, 13, 17, 18, 32, 33, 43, 47, 48, 53, 56, 63, 67, 76, 82, 83, 92, 95, 102, 106, 113, 115, 125, 129, 136, 140, 141, 146, 150, 162, 164, 165, 167, 169, 171, 172, 173, 181, 186, 187, 194, 195, 198, 203, 204, 207, 208, 212, 216, 217, 222, 223, 228, 229, 233, 236, 238, 243, 244, 246, 248, 252, 255, 257, 259, 260, 267, 268, 272, 276, 277, 278, 280, 281, 284, 293, 295, 297, 300, 310, 316, 321, 322, 323, 324, 325, 327, 330, 338, 343, 347, 348, 350, 353, 355, 356, 358, 361, 362, 364, 375, 377, 379, 381, 382, 384, 391, 392, 393, 396, 398, 3

In [18]:
log_header.stats

EvalStats(started_at='2025-02-13T17:08:02+00:00', completed_at='2025-02-13T17:21:43+00:00', model_usage={'openai/o1-mini-2024-09-12': ModelUsage(input_tokens=988900, output_tokens=9914854, total_tokens=10903754, input_tokens_cache_write=None, input_tokens_cache_read=None), 'google/gemini-1.5-flash-002': ModelUsage(input_tokens=1765867, output_tokens=10580, total_tokens=1776447, input_tokens_cache_write=None, input_tokens_cache_read=None)})

In [19]:
log_header.eval.model

'openai/o1-mini-2024-09-12'

In [20]:
model = log_header.eval.model
log_header.stats.model_usage[model]

ModelUsage(input_tokens=988900, output_tokens=9914854, total_tokens=10903754, input_tokens_cache_write=None, input_tokens_cache_read=None)

In [21]:
log_header.eval

EvalSpec(run_id='4DWnpJowAfeQDHc2xDNy2C', created='2025-02-13T17:08:02+00:00', task='MATH level 5', task_id='ZAhShmdDBAB9WnsZdx5qHf', task_version=0, task_file='bench/task/hendrycks_math/__init__.py', task_attribs={}, task_args={}, solver=None, solver_args=None, tags=None, dataset=EvalDataset(name='tadamcz/hendrycks___competition_math', location='tadamcz/hendrycks___competition_math', samples=1324, sample_ids=[1, 2, 12, 13, 17, 18, 32, 33, 43, 47, 48, 53, 56, 63, 67, 76, 82, 83, 92, 95, 102, 106, 113, 115, 125, 129, 136, 140, 141, 146, 150, 162, 164, 165, 167, 169, 171, 172, 173, 181, 186, 187, 194, 195, 198, 203, 204, 207, 208, 212, 216, 217, 222, 223, 228, 229, 233, 236, 238, 243, 244, 246, 248, 252, 255, 257, 259, 260, 267, 268, 272, 276, 277, 278, 280, 281, 284, 293, 295, 297, 300, 310, 316, 321, 322, 323, 324, 325, 327, 330, 338, 343, 347, 348, 350, 353, 355, 356, 358, 361, 362, 364, 375, 377, 379, 381, 382, 384, 391, 392, 393, 396, 398, 399, 400, 403, 406, 407, 410, 412, 416, 417

In [22]:
log_header.eval.task

'MATH level 5'

In [23]:
log_header.plan.config.reasoning_effort

'high'

In [24]:
log_header.eval.config.epochs

4

In [25]:
log_header.results.total_samples

5296

In [26]:
eval_data = []
with open('data/epoch_ai_eval_log_headers.txt', 'w') as f:
    for run in runs:
        log_header = read_eval_log(convert_to_s3_path(run.logs), header_only=True)
        f.write(str(log_header))
        f.write('\n\n')
        model_path = log_header.eval.model
        model_name = model_path.split('/')[-1]
        task = log_header.eval.task
        reasoning_effort = log_header.plan.config.reasoning_effort
        if reasoning_effort is not None:
            model_name += '_high'
        epochs = log_header.eval.config.epochs
        total_samples = log_header.results.total_samples
        model_usage = log_header.stats.model_usage[model_path]
        eval_data.append({
            'Model': model_name,
            'Task': task,
            'epochs': epochs,
            'total_samples': total_samples,
            'input_tokens': model_usage.input_tokens,
            'output_tokens': model_usage.output_tokens,
            'total_tokens': model_usage.total_tokens,
        })
        print(model_path, model_usage)
df = pd.DataFrame(eval_data)
df.to_csv('data/epoch_ai_eval_data.csv', index=False)

sagemaker/01-ai/Yi-34B-Chat input_tokens=2222376 output_tokens=7308455 total_tokens=9530831 input_tokens_cache_write=None input_tokens_cache_read=None
fireworks/accounts/fireworks/models/qwen2p5-72b-instruct input_tokens=2243320 output_tokens=9708977 total_tokens=11952297 input_tokens_cache_write=None input_tokens_cache_read=None
openai/gpt-4-0613 input_tokens=1982888 output_tokens=3620454 total_tokens=5603342 input_tokens_cache_write=None input_tokens_cache_read=None
openai/o1-mini-2024-09-12 input_tokens=988900 output_tokens=9914854 total_tokens=10903754 input_tokens_cache_write=None input_tokens_cache_read=None
sagemaker/google/gemma-2-9b-it input_tokens=1989896 output_tokens=4551564 total_tokens=6541460 input_tokens_cache_write=None input_tokens_cache_read=None
sagemaker/microsoft/phi-4 input_tokens=838784 output_tokens=2106559 total_tokens=2945343 input_tokens_cache_write=None input_tokens_cache_read=None
mistral/mistral-large-2402 input_tokens=928128 output_tokens=1152988 total_t

In [27]:
df

,Model,Task,epochs,total_samples,input_tokens,output_tokens,total_tokens
0,Yi-34B-Chat,MATH level 5,8,10592,2222376,7308455,9530831
1,qwen2p5-72b-instruct,MATH level 5,8,10592,2243320,9708977,11952297
2,gpt-4-0613,MATH level 5,8,10592,1982888,3620454,5603342
3,o1-mini-2024-09-12_high,MATH level 5,4,5296,988900,9914854,10903754
4,gemma-2-9b-it,MATH level 5,8,10592,1989896,4551564,6541460
...,...,...,...,...,...,...,...
130,open-mistral-7b,MATH level 5,8,10592,2150624,6429532,8580156
131,Qwen2-72B-Instruct,GPQA Diamond,16,3168,889664,1028460,1918124
132,claude-3-5-sonnet-20240620,MATH level 5,8,10592,2137040,4674647,6811687
133,claude-3-opus-20240229,MATH level 5,8,10592,2137040,5204915,7341955
